In [476]:
#Importing Necessary Libraries
import pandas as pd
import numpy as np

import pymysql
from sqlalchemy import create_engine

In [477]:
#Fetching Search output as Dataframe for further cleaning
df = pd.read_json(r'search_output.json')

In [478]:
df

,title,url,rating,review_count,price
0,Simple Mobile Samsung Galaxy A01 4G LTE Prepai...,https://www.amazon.com/gp/slredirect/picassoRe...,4.3 out of 5 stars,372,$53.39
1,Simple Mobile Samsung Galaxy A20 4G LTE Prepai...,https://www.amazon.com/gp/slredirect/picassoRe...,4.4 out of 5 stars,30,$144.66
2,Samsung Galaxy A21 Factory Unlocked Android Ce...,https://www.amazon.com/Samsung-Unlocked-Smartp...,4.4 out of 5 stars,659,$99.00
3,"OUKITEL WP8 Pro (2020) Rugged Smartphone, 6.49...",https://www.amazon.com/OUKITEL-WP8-Pro-Smartph...,4.4 out of 5 stars,271,$189.99
4,Samsung Galaxy S20 FE 5G | Factory Unlocked An...,https://www.amazon.com/Samsung-Factory-Unlocke...,4.6 out of 5 stars,"2,265",$599.99
...,...,...,...,...,...
6631,Rechargeable Selfie Camera Ring Light - Portab...,https://www.amazon.com/Rechargeable-Selfie-Cam...,4.1 out of 5 stars,34,$7.99
6632,PAKPOD Bundle - Packable Tripod for Mirrorless...,https://www.amazon.com/PAKPOD-BUNDLE-Mirrorles...,5.0 out of 5 stars,3,$99.00
6633,Swark OLED Display Compatible with Google Pixe...,https://www.amazon.com/Display-Compatible-Smar...,3.8 out of 5 stars,22,$188.00
6634,Tracfone TCL A1X 4G LTE Prepaid Smartphone (Lo...,https://www.amazon.com/gp/slredirect/picassoRe...,4.1 out of 5 stars,62,$59.99


In [479]:
#Quick check on Null Values(if any)
df.isnull().sum()

title             0
url               0
rating          734
review_count    734
price           111
dtype: int64

In [480]:
df.dropna(inplace= True)

In [481]:
#Cleaning Rating column in order to just get 'The Rating' of the product
df['rating'].map(lambda x:x.split(' ', 1)[0])

0       4.3
1       4.4
2       4.4
3       4.4
4       4.6
       ... 
6630    4.1
6631    4.1
6632    5.0
6633    3.8
6634    4.1
Name: rating, Length: 5794, dtype: object

In [482]:
df.rating = df['rating'].map(lambda x:x.split(' ', 1)[0])
df.head()

,title,url,rating,review_count,price
0,Simple Mobile Samsung Galaxy A01 4G LTE Prepai...,https://www.amazon.com/gp/slredirect/picassoRe...,4.3,372,$53.39
1,Simple Mobile Samsung Galaxy A20 4G LTE Prepai...,https://www.amazon.com/gp/slredirect/picassoRe...,4.4,30,$144.66
2,Samsung Galaxy A21 Factory Unlocked Android Ce...,https://www.amazon.com/Samsung-Unlocked-Smartp...,4.4,659,$99.00
3,"OUKITEL WP8 Pro (2020) Rugged Smartphone, 6.49...",https://www.amazon.com/OUKITEL-WP8-Pro-Smartph...,4.4,271,$189.99
4,Samsung Galaxy S20 FE 5G | Factory Unlocked An...,https://www.amazon.com/Samsung-Factory-Unlocke...,4.6,"2,265",$599.99


In [483]:
#Converting column 'Rating' as Float
df.rating = df.rating.astype(float)

In [484]:
df['review_count'] = df['review_count'].str.replace(',', '').astype(int)

In [485]:
df.dtypes

title            object
url              object
rating          float64
review_count      int32
price            object
dtype: object

In [486]:
#Removing '$' from Price column
df['price'] = df['price'].str.replace('$', '').str.replace(',', '').astype(float)

In [487]:
df.head()

,title,url,rating,review_count,price
0,Simple Mobile Samsung Galaxy A01 4G LTE Prepai...,https://www.amazon.com/gp/slredirect/picassoRe...,4.3,372,53.39
1,Simple Mobile Samsung Galaxy A20 4G LTE Prepai...,https://www.amazon.com/gp/slredirect/picassoRe...,4.4,30,144.66
2,Samsung Galaxy A21 Factory Unlocked Android Ce...,https://www.amazon.com/Samsung-Unlocked-Smartp...,4.4,659,99.00
3,"OUKITEL WP8 Pro (2020) Rugged Smartphone, 6.49...",https://www.amazon.com/OUKITEL-WP8-Pro-Smartph...,4.4,271,189.99
4,Samsung Galaxy S20 FE 5G | Factory Unlocked An...,https://www.amazon.com/Samsung-Factory-Unlocke...,4.6,2265,599.99


In [488]:
#Adding column 'Product_Type' and name it as smartphone as we would be dealing only with Smartphone product
df['product_type'] = 'smartphone'
df.head()

,title,url,rating,review_count,price,product_type
0,Simple Mobile Samsung Galaxy A01 4G LTE Prepai...,https://www.amazon.com/gp/slredirect/picassoRe...,4.3,372,53.39,smartphone
1,Simple Mobile Samsung Galaxy A20 4G LTE Prepai...,https://www.amazon.com/gp/slredirect/picassoRe...,4.4,30,144.66,smartphone
2,Samsung Galaxy A21 Factory Unlocked Android Ce...,https://www.amazon.com/Samsung-Unlocked-Smartp...,4.4,659,99.00,smartphone
3,"OUKITEL WP8 Pro (2020) Rugged Smartphone, 6.49...",https://www.amazon.com/OUKITEL-WP8-Pro-Smartph...,4.4,271,189.99,smartphone
4,Samsung Galaxy S20 FE 5G | Factory Unlocked An...,https://www.amazon.com/Samsung-Factory-Unlocke...,4.6,2265,599.99,smartphone


In [489]:
#Resetting Index
df_new = df.reset_index(drop=True)

In [490]:
#Getting URLs from the dataset
urls = df_new['url']
urls.head()

0    https://www.amazon.com/gp/slredirect/picassoRe...
1    https://www.amazon.com/gp/slredirect/picassoRe...
2    https://www.amazon.com/Samsung-Unlocked-Smartp...
3    https://www.amazon.com/OUKITEL-WP8-Pro-Smartph...
4    https://www.amazon.com/Samsung-Factory-Unlocke...
Name: url, dtype: object

In [491]:
#Splitting column 'URL' in different columns in order to fetch the Unique number known as ASIN for Amazon
cols = ['c1', 'c2', 'c3', 'c4','c5','c6']
make_me = []
for url in urls:
    lst = url.split("/")
    # your business rules go here
    make_me.append([x for x in lst if not x.isdigit() and not x == ""])
    

df8 = pd.DataFrame(make_me, columns=cols)
df8.head(10)

,c1,c2,c3,c4,c5,c6
0,https:,www.amazon.com,gp,slredirect,picassoRedirect.html,ref=pa_sp_atf_electronics-intl-ship_sr_pg1_1?i...
1,https:,www.amazon.com,gp,slredirect,picassoRedirect.html,ref=pa_sp_atf_electronics-intl-ship_sr_pg1_1?i...
2,https:,www.amazon.com,Samsung-Unlocked-Smartphone-Long-Lasting-Infinity,dp,B08DHG8SBB,ref=sr_1_3?crid=17OSYBMPPLBAI&dchild=1&keyword...
3,https:,www.amazon.com,OUKITEL-WP8-Pro-Smartphone-Fingerprint,dp,B08HRJM99F,ref=sr_1_4?crid=17OSYBMPPLBAI&dchild=1&keyword...
4,https:,www.amazon.com,Samsung-Factory-Unlocked-Smartphone-Pro-Grade,dp,B08FYTSXGQ,ref=sr_1_5?crid=17OSYBMPPLBAI&dchild=1&keyword...
5,https:,www.amazon.com,BLU-G90-Pro-Gaming-Smartphone,dp,B08C6XBCRT,ref=sr_1_6?crid=17OSYBMPPLBAI&dchild=1&keyword...
6,https:,www.amazon.com,gp,slredirect,picassoRedirect.html,ref=pa_sp_mtf_electronics-intl-ship_sr_pg1_1?i...
7,https:,www.amazon.com,DJI-OM-Handheld-Smartphone-Stabilizer,dp,B08CMVGVQY,ref=sr_1_8?crid=17OSYBMPPLBAI&dchild=1&keyword...
8,https:,www.amazon.com,TCL-Unlocked-Android-Smartphone-Display,dp,B087LY84RT,ref=sr_1_9?crid=17OSYBMPPLBAI&dchild=1&keyword...
9,https:,www.amazon.com,Motorola-XT2041-1-Unlocked-Android-Smartphone,dp,B087MZ1GZB,ref=sr_1_10?crid=17OSYBMPPLBAI&dchild=1&keywor...


In [492]:
#Picking only ASIN column value
abc = df8['c5']
abc.head()

0    picassoRedirect.html
1    picassoRedirect.html
2              B08DHG8SBB
3              B08HRJM99F
4              B08FYTSXGQ
Name: c5, dtype: object

In [493]:
df2 = df8['c6']
df2.head()

0    ref=pa_sp_atf_electronics-intl-ship_sr_pg1_1?i...
1    ref=pa_sp_atf_electronics-intl-ship_sr_pg1_1?i...
2    ref=sr_1_3?crid=17OSYBMPPLBAI&dchild=1&keyword...
3    ref=sr_1_4?crid=17OSYBMPPLBAI&dchild=1&keyword...
4    ref=sr_1_5?crid=17OSYBMPPLBAI&dchild=1&keyword...
Name: c6, dtype: object

In [494]:
cols = ['c1','c2','c3','d4','c5','c6','c7','c8','c9','c10','c11','c12','c13','c14', 'c15']
make_me = []
for url in urls:
    lst = url.split("%2")
    # your business rules go here
    make_me.append([x for x in lst if not x.isdigit() and not x == ""])
    

df4 = pd.DataFrame(make_me, columns=cols)
df4

,c1,c2,c3,d4,c5,c6,c7,c8,c9,c10,c11,c12,c13,c14,c15
0,https://www.amazon.com/gp/slredirect/picassoRe...,FSimple-Mobile-Samsung-Prepaid-Smartphone,Fdp,FB088N774B7,Fref%3Dsr_1_1_sspa%3Fcrid%3D17OSYBMPPLBAI,6dchild%3D1,6keywords%3Dsmartphone,6qid%3D1611430937,6s%3Delectronics,6sprefix%3Dsmart,52Celectronics-intl-ship,52C250,6sr%3D1-1-spons,6psc%3D1&qualifier=1611430937&id=8516783865991...,None
1,https://www.amazon.com/gp/slredirect/picassoRe...,FSimple-Mobile-Samsung-Prepaid-Smartphone,Fdp,FB07Y5V9CSW,Fref%3Dsr_1_2_sspa%3Fcrid%3D17OSYBMPPLBAI,6dchild%3D1,6keywords%3Dsmartphone,6qid%3D1611430937,6s%3Delectronics,6sprefix%3Dsmart,52Celectronics-intl-ship,52C250,6sr%3D1-2-spons,6psc%3D1&qualifier=1611430937&id=8516783865991...,None
2,https://www.amazon.com/Samsung-Unlocked-Smartp...,Celectronics-intl-ship,C250&sr=1-3,None,None,None,None,None,None,None,None,None,None,None,None
3,https://www.amazon.com/OUKITEL-WP8-Pro-Smartph...,Celectronics-intl-ship,C250&sr=1-4,None,None,None,None,None,None,None,None,None,None,None,None
4,https://www.amazon.com/Samsung-Factory-Unlocke...,Celectronics-intl-ship,C250&sr=1-5,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5789,https://www.amazon.com/Sevenoak-SK-R01-Shoulde...,Celectronics-intl-ship,C250&sr=1-4429,None,None,None,None,None,None,None,None,None,None,None,None
5790,https://www.amazon.com/Rechargeable-Selfie-Cam...,Celectronics-intl-ship,C250&sr=1-4430,None,None,None,None,None,None,None,None,None,None,None,None
5791,https://www.amazon.com/PAKPOD-BUNDLE-Mirrorles...,Celectronics-intl-ship,C250&sr=1-4431,None,None,None,None,None,None,None,None,None,None,None,None
5792,https://www.amazon.com/Display-Compatible-Smar...,Celectronics-intl-ship,C250&sr=1-4432,None,None,None,None,None,None,None,None,None,None,None,None


In [495]:
abcd = df4['d4']
abcd

0       FB088N774B7
1       FB07Y5V9CSW
2              None
3              None
4              None
           ...     
5789           None
5790           None
5791           None
5792           None
5793    FB07Y5V385Y
Name: d4, Length: 5794, dtype: object

In [496]:
pd.DataFrame(abcd)

,d4
0,FB088N774B7
1,FB07Y5V9CSW
2,None
3,None
4,None
...,...
5789,None
5790,None
5791,None
5792,None


In [497]:
#Merging 2 datasets in order t fetch all ASIN numbers
abcde = pd.concat([abc, abcd], axis=1)
abcde

,c5,d4
0,picassoRedirect.html,FB088N774B7
1,picassoRedirect.html,FB07Y5V9CSW
2,B08DHG8SBB,None
3,B08HRJM99F,None
4,B08FYTSXGQ,None
...,...,...
5789,B0079X63BQ,None
5790,B088Z5FPMB,None
5791,B01M8NUQWK,None
5792,B07BGYRFYP,None


In [498]:
abcde['d4'] = abcde['d4'].str[1:]
abcde

,c5,d4
0,picassoRedirect.html,B088N774B7
1,picassoRedirect.html,B07Y5V9CSW
2,B08DHG8SBB,None
3,B08HRJM99F,None
4,B08FYTSXGQ,None
...,...,...
5789,B0079X63BQ,None
5790,B088Z5FPMB,None
5791,B01M8NUQWK,None
5792,B07BGYRFYP,None


In [499]:
abcde['c5'] = np.where(abcde['c5'] == 'picassoRedirect.html', abcde['d4'], abcde['c5'])
abcde

,c5,d4
0,B088N774B7,B088N774B7
1,B07Y5V9CSW,B07Y5V9CSW
2,B08DHG8SBB,None
3,B08HRJM99F,None
4,B08FYTSXGQ,None
...,...,...
5789,B0079X63BQ,None
5790,B088Z5FPMB,None
5791,B01M8NUQWK,None
5792,B07BGYRFYP,None


In [500]:
#Here is the new column with all ASIN(Unque number attached to every product)
new_f1 = abcde.drop(['d4',], axis=1)
new_f1

,c5
0,B088N774B7
1,B07Y5V9CSW
2,B08DHG8SBB
3,B08HRJM99F
4,B08FYTSXGQ
...,...
5789,B0079X63BQ
5790,B088Z5FPMB
5791,B01M8NUQWK
5792,B07BGYRFYP


In [501]:
#Including ASIN(Unique Number) to the main data frame
final_1 = pd.concat([df_new, new_f1], axis=1)
final_1

,title,url,rating,review_count,price,product_type,c5
0,Simple Mobile Samsung Galaxy A01 4G LTE Prepai...,https://www.amazon.com/gp/slredirect/picassoRe...,4.3,372,53.39,smartphone,B088N774B7
1,Simple Mobile Samsung Galaxy A20 4G LTE Prepai...,https://www.amazon.com/gp/slredirect/picassoRe...,4.4,30,144.66,smartphone,B07Y5V9CSW
2,Samsung Galaxy A21 Factory Unlocked Android Ce...,https://www.amazon.com/Samsung-Unlocked-Smartp...,4.4,659,99.00,smartphone,B08DHG8SBB
3,"OUKITEL WP8 Pro (2020) Rugged Smartphone, 6.49...",https://www.amazon.com/OUKITEL-WP8-Pro-Smartph...,4.4,271,189.99,smartphone,B08HRJM99F
4,Samsung Galaxy S20 FE 5G | Factory Unlocked An...,https://www.amazon.com/Samsung-Factory-Unlocke...,4.6,2265,599.99,smartphone,B08FYTSXGQ
...,...,...,...,...,...,...,...
5789,Handheld Stabilizer & Video Led Lights & Remot...,https://www.amazon.com/Sevenoak-SK-R01-Shoulde...,4.1,37,54.99,smartphone,B0079X63BQ
5790,Rechargeable Selfie Camera Ring Light - Portab...,https://www.amazon.com/Rechargeable-Selfie-Cam...,4.1,34,7.99,smartphone,B088Z5FPMB
5791,PAKPOD Bundle - Packable Tripod for Mirrorless...,https://www.amazon.com/PAKPOD-BUNDLE-Mirrorles...,5.0,3,99.00,smartphone,B01M8NUQWK
5792,Swark OLED Display Compatible with Google Pixe...,https://www.amazon.com/Display-Compatible-Smar...,3.8,22,188.00,smartphone,B07BGYRFYP


In [502]:
#Dropping URL from the dataset
new_f2 = final_1.drop(['url',], axis=1)
new_f2

,title,rating,review_count,price,product_type,c5
0,Simple Mobile Samsung Galaxy A01 4G LTE Prepai...,4.3,372,53.39,smartphone,B088N774B7
1,Simple Mobile Samsung Galaxy A20 4G LTE Prepai...,4.4,30,144.66,smartphone,B07Y5V9CSW
2,Samsung Galaxy A21 Factory Unlocked Android Ce...,4.4,659,99.00,smartphone,B08DHG8SBB
3,"OUKITEL WP8 Pro (2020) Rugged Smartphone, 6.49...",4.4,271,189.99,smartphone,B08HRJM99F
4,Samsung Galaxy S20 FE 5G | Factory Unlocked An...,4.6,2265,599.99,smartphone,B08FYTSXGQ
...,...,...,...,...,...,...
5789,Handheld Stabilizer & Video Led Lights & Remot...,4.1,37,54.99,smartphone,B0079X63BQ
5790,Rechargeable Selfie Camera Ring Light - Portab...,4.1,34,7.99,smartphone,B088Z5FPMB
5791,PAKPOD Bundle - Packable Tripod for Mirrorless...,5.0,3,99.00,smartphone,B01M8NUQWK
5792,Swark OLED Display Compatible with Google Pixe...,3.8,22,188.00,smartphone,B07BGYRFYP


In [503]:
#Renaming 
new_f3 =new_f2.rename(columns={'c5': 'ASIN'})
new_f3

,title,rating,review_count,price,product_type,ASIN
0,Simple Mobile Samsung Galaxy A01 4G LTE Prepai...,4.3,372,53.39,smartphone,B088N774B7
1,Simple Mobile Samsung Galaxy A20 4G LTE Prepai...,4.4,30,144.66,smartphone,B07Y5V9CSW
2,Samsung Galaxy A21 Factory Unlocked Android Ce...,4.4,659,99.00,smartphone,B08DHG8SBB
3,"OUKITEL WP8 Pro (2020) Rugged Smartphone, 6.49...",4.4,271,189.99,smartphone,B08HRJM99F
4,Samsung Galaxy S20 FE 5G | Factory Unlocked An...,4.6,2265,599.99,smartphone,B08FYTSXGQ
...,...,...,...,...,...,...
5789,Handheld Stabilizer & Video Led Lights & Remot...,4.1,37,54.99,smartphone,B0079X63BQ
5790,Rechargeable Selfie Camera Ring Light - Portab...,4.1,34,7.99,smartphone,B088Z5FPMB
5791,PAKPOD Bundle - Packable Tripod for Mirrorless...,5.0,3,99.00,smartphone,B01M8NUQWK
5792,Swark OLED Display Compatible with Google Pixe...,3.8,22,188.00,smartphone,B07BGYRFYP


In [504]:
cols = new_f3.columns.tolist()
cols

['title', 'rating', 'review_count', 'price', 'product_type', 'ASIN']

In [505]:
cols = cols[-1:] + cols[:-1]
cols

['ASIN', 'title', 'rating', 'review_count', 'price', 'product_type']

In [506]:
new_f3 = new_f3[cols]
new_f3

,ASIN,title,rating,review_count,price,product_type
0,B088N774B7,Simple Mobile Samsung Galaxy A01 4G LTE Prepai...,4.3,372,53.39,smartphone
1,B07Y5V9CSW,Simple Mobile Samsung Galaxy A20 4G LTE Prepai...,4.4,30,144.66,smartphone
2,B08DHG8SBB,Samsung Galaxy A21 Factory Unlocked Android Ce...,4.4,659,99.00,smartphone
3,B08HRJM99F,"OUKITEL WP8 Pro (2020) Rugged Smartphone, 6.49...",4.4,271,189.99,smartphone
4,B08FYTSXGQ,Samsung Galaxy S20 FE 5G | Factory Unlocked An...,4.6,2265,599.99,smartphone
...,...,...,...,...,...,...
5789,B0079X63BQ,Handheld Stabilizer & Video Led Lights & Remot...,4.1,37,54.99,smartphone
5790,B088Z5FPMB,Rechargeable Selfie Camera Ring Light - Portab...,4.1,34,7.99,smartphone
5791,B01M8NUQWK,PAKPOD Bundle - Packable Tripod for Mirrorless...,5.0,3,99.00,smartphone
5792,B07BGYRFYP,Swark OLED Display Compatible with Google Pixe...,3.8,22,188.00,smartphone


In [507]:
#Finding the best reviewed product with maximum rating. 
best_1 = new_f3.sort_values(['review_count','rating'], ascending= False)
best_1

,ASIN,title,rating,review_count,price,product_type
3586,B0887CHVFF,SAMSUNG EVO Select 512GB microSDXC UHS-I U3 10...,4.8,99921,64.99,smartphone
2765,B07TWFWJDZ,Fitbit Versa 2 Health and Fitness Smartwatch w...,4.6,84420,177.00,smartphone
5754,B00R2AZLD2,"NETGEAR Nighthawk Smart Wi-Fi Router, R6700 - ...",4.3,66736,76.49,smartphone
4697,B019U00D7K,COWIN E7 Active Noise Cancelling Headphones Bl...,4.4,62473,69.99,smartphone
1301,B01KZC1EAW,Syncwire Long Aux Cable 6.5Ft- Auxiliary Audio...,4.6,50605,11.99,smartphone
...,...,...,...,...,...,...
3524,B07HMP8K11,PHONSUN USB Charging Flex Cable w Microphone R...,1.0,1,13.49,smartphone
3531,B07HMP8K11,PHONSUN USB Charging Flex Cable w Microphone R...,1.0,1,13.49,smartphone
3680,B00XV0BYME,"Samsung Galaxy S6 EDGE G925v 32GB 5"" AMOLED Ve...",1.0,1,199.99,smartphone
5426,B08PYJK6XT,Pocket Printer - Wireless Mini Photo Printer f...,1.0,1,49.99,smartphone


In [508]:
#Saving it to CSV
new_f3.to_csv('search_smartphone.csv')

In [509]:
#Fetching Product Output from the folder
df1 = pd.read_json(r'product_output.json')
df1.head()

,name,product_tech_spec,seller,seller_link,freq_bought,freq_bought_link,link_to_all_reviews
0,Simple Mobile Samsung Galaxy A01 4G LTE Prepai...,"[{'info': 'Product Dimensions', 'value': '6.75...",None,None,Simple Mobile LG Journey 4G LTE Prepaid Smartp...,/Simple-Mobile-Journey-Prepaid-Smartphone/dp/B...,/Simple-Mobile-Samsung-Prepaid-Smartphone/prod...
1,Simple Mobile Samsung Galaxy A20 4G LTE Prepai...,"[{'info': 'Product Dimensions', 'value': '7.25...",None,None,[3-PACK]-Mr.Shield For Samsung Galaxy A20 [Not...,/Mr-Shield-Samsung-Galaxy-A20-Tempered-Protect...,/Simple-Mobile-Samsung-Prepaid-Smartphone/prod...
2,None,None,None,None,None,None,None
3,"OUKITEL WP8 Pro (2020) Rugged Smartphone, 6.49...","[{'info': 'Product Dimensions', 'value': '7.01...",Moon and Six Pence,https://www.amazon.com/gp/help/seller/at-a-gla...,SAMSUNG: EVO Select 128GB MicroSDXC UHS-I U3 1...,https://www.amazon.com/SAMSUNG-Select-microSDX...,/OUKITEL-WP8-Pro-Smartphone-Fingerprint/produc...
4,Samsung Galaxy S20 FE 5G | Factory Unlocked An...,"[{'info': 'Product Dimensions', 'value': '6.7 ...",None,None,None,None,/Samsung-Factory-Unlocked-Smartphone-Pro-Grade...


In [510]:
#Quick check on Null values
df1.isnull().sum()

name                     33
product_tech_spec       139
seller                 1730
seller_link            1730
freq_bought            2853
freq_bought_link       2853
link_to_all_reviews     920
dtype: int64

In [511]:
#Checking Product Specification of fetched products
p1 = df1[['name', 'product_tech_spec']]
p1

,name,product_tech_spec
0,Simple Mobile Samsung Galaxy A01 4G LTE Prepai...,"[{'info': 'Product Dimensions', 'value': '6.75..."
1,Simple Mobile Samsung Galaxy A20 4G LTE Prepai...,"[{'info': 'Product Dimensions', 'value': '7.25..."
2,None,None
3,"OUKITEL WP8 Pro (2020) Rugged Smartphone, 6.49...","[{'info': 'Product Dimensions', 'value': '7.01..."
4,Samsung Galaxy S20 FE 5G | Factory Unlocked An...,"[{'info': 'Product Dimensions', 'value': '6.7 ..."
...,...,...
5528,Samsung Galaxy S20 FE 5G | Factory Unlocked An...,"[{'info': 'Product Dimensions', 'value': '6.7 ..."
5529,Gimbal Stabilizer 3-axis Handheld Portable Vlo...,"[{'info': 'Package Dimensions', 'value': '8.46..."
5530,Godox X2T-S 2.4G TTL Wireless Flash Trigger wi...,"[{'info': 'Package Dimensions', 'value': '7.72..."
5531,"Car Phone Mount,Cell Phone Holder for Car,Anti...","[{'info': 'Package Dimensions', 'value': '4.9 ..."


In [512]:
p1.product_tech_spec[0]

[{'info': 'Product Dimensions', 'value': '6.75 x 2.5 x 10.38 inches'},
 {'info': 'Item Weight', 'value': '14.4 ounces'},
 {'info': 'Domestic Shipping', 'value': 'Item can be shipped within U.S.'},
 {'info': 'International Shipping',
  'value': 'This item can be shipped to select countries outside of the U.S. Learn More'},
 {'info': 'ASIN', 'value': 'B088N774B7'},
 {'info': 'Item model number', 'value': 'SMSAS111DGP5'},
 {'info': 'Batteries',
  'value': '1 Lithium ion batteries required. (included)'},
 {'info': 'Customer Reviews',
  'value': '/*\n* Fix for UDP-1061. Average customer reviews has a small extra line on hover\n* https://omni-grok.amazon.com/xref/src/appgroup/websiteTemplates/retail/SoftlinesDetailPageAssets/udp-intl-lock/src/legacy.css?indexName=WebsiteTemplates#40\n*/\n.noUnderline a:hover {\ntext-decoration: none;\n} 4.3 out of 5 stars 379 ratings P.when(\'A\', \'ready\').execute(function(A) {\nA.declarative(\'acrLink-click-metrics\', \'click\', { "allowLinkDefault" : tru

In [513]:
#Splitting info section from the product specification
add_col =  []
for i in range(p1.shape[0]):
    try:
        for j in range(len(p1.product_tech_spec[i])):
            col = p1.product_tech_spec[i][j]['info']
            if col not in add_col:
                add_col.append(col)
    except:
        continue

In [514]:
add_col

['Product Dimensions',
 'Item Weight',
 'Domestic Shipping',
 'International Shipping',
 'ASIN',
 'Item model number',
 'Batteries',
 'Customer Reviews',
 'Best Sellers Rank',
 'OS',
 'GPS',
 'Other display features',
 'Device interface - primary',
 'Other camera features',
 'Form Factor',
 'Manufacturer',
 'Date First Available',
 'RAM',
 'Connectivity technologies',
 'Included Components',
 'Wireless communication technologies',
 'Display technology',
 'Colour',
 'Battery Power Rating',
 'Is Discontinued By Manufacturer',
 'Language:',
 'Additional Features',
 'Package Dimensions',
 'Scanner Resolution',
 'Audio Jack',
 'Phone Talk Time',
 'Color Name',
 'Hardware Platform',
 'Department',
 'Compatible Devices',
 'Material Type',
 'Size',
 'Connector Type',
 'Power Source',
 'Phone Standy Time (with data)',
 'Battery type',
 'Display resolution',
 'Warranty Description',
 'Batteries Required',
 'Import Designation',
 'Body Material',
 'Hardware Interface',
 'Supported Software',
 'Sp

In [515]:
#Adding name of the product to the dataset
m_1 = pd.DataFrame(columns=['name'] + add_col)
m_1

,name,Product Dimensions,Item Weight,Domestic Shipping,International Shipping,ASIN,Item model number,Batteries,Customer Reviews,Best Sellers Rank,...,Is Assembly Required?,Manufacturer recommended age,Release date,Mfg Recommended age,Compatible Phone Models,Standing screen display size,GPU,Specific Uses For Product,Flash,Finish Types


In [516]:
for i in range(p1.shape[0]):
    try:
        for j in range(len(p1.product_tech_spec[i])):
            col = p1.product_tech_spec[i][j]['info']
            val = p1.product_tech_spec[i][j]['value']

            m_1.loc[i,col] = val
            
    except:
        m_1.loc[i,col] = ''
        continue

In [517]:
m_1.name = p1.name
m_1.head()

,name,Product Dimensions,Item Weight,Domestic Shipping,International Shipping,ASIN,Item model number,Batteries,Customer Reviews,Best Sellers Rank,...,Is Assembly Required?,Manufacturer recommended age,Release date,Mfg Recommended age,Compatible Phone Models,Standing screen display size,GPU,Specific Uses For Product,Flash,Finish Types
0,Simple Mobile Samsung Galaxy A01 4G LTE Prepai...,6.75 x 2.5 x 10.38 inches,14.4 ounces,Item can be shipped within U.S.,This item can be shipped to select countries o...,B088N774B7,SMSAS111DGP5,1 Lithium ion batteries required. (included),/*\n* Fix for UDP-1061. Average customer revie...,"#3,750 in Cell Phones & Accessories ( See Top ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Simple Mobile Samsung Galaxy A20 4G LTE Prepai...,7.25 x 1.65 x 10.25 inches,15.2 ounces,Item can be shipped within U.S.,This item can be shipped to select countries o...,B07Y5V9CSW,SMSAS205DGP5,1 Lithium ion batteries required. (included),/*\n* Fix for UDP-1061. Average customer revie...,"#45,762 in Cell Phones & Accessories ( See Top...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"OUKITEL WP8 Pro (2020) Rugged Smartphone, 6.49...",7.01 x 3.33 x 0.51 inches,1.23 pounds,NaN,NaN,B08HRJM99F,WP8 Pro,1 Lithium Polymer batteries required. (included),/*\n* Fix for UDP-1061. Average customer revie...,"#9,229 in Cell Phones & Accessories ( See Top ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Samsung Galaxy S20 FE 5G | Factory Unlocked An...,6.7 x 3.4 x 2.44 inches,6.7 ounces,NaN,NaN,B08FYTSXGQ,SM-G781UZBMXAA,1 Lithium ion batteries required. (included),/*\n* Fix for UDP-1061. Average customer revie...,#491 in Cell Phones & Accessories ( See Top 10...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [518]:
m_1.isnull().sum().tail(40)

Package Dimensions               3664
Scanner Resolution               5408
Audio Jack                       5461
Phone Talk Time                  5483
Color Name                       5484
Hardware Platform                5469
Department                       5442
Compatible Devices               5529
Material Type                    5512
Size                             5506
Connector Type                   5474
Power Source                     5498
Phone Standy Time (with data)    5515
Battery type                     5514
Display resolution               5496
Warranty Description             5528
Batteries Required               5514
Import Designation               5530
Body Material                    5530
Hardware Interface               5532
Supported Software               5532
Speaker Count                    5532
Output Wattage                   5532
Wireless Microphone Frequency    5531
GSM frequencies                  5532
Color Screen                     5528
Voltage     

In [519]:
for col_name in m_1.columns:
    print(col_name)

name
Product Dimensions
Item Weight
Domestic Shipping
International Shipping
ASIN
Item model number
Batteries
Customer Reviews
Best Sellers Rank
OS
GPS
Other display features
Device interface - primary
Other camera features
Form Factor
Manufacturer
Date First Available
RAM
Connectivity technologies
Included Components
Wireless communication technologies
Display technology
Colour
Battery Power Rating
Is Discontinued By Manufacturer
Language:
Additional Features
Package Dimensions
Scanner Resolution
Audio Jack
Phone Talk Time
Color Name
Hardware Platform
Department
Compatible Devices
Material Type
Size
Connector Type
Power Source
Phone Standy Time (with data)
Battery type
Display resolution
Warranty Description
Batteries Required
Import Designation
Body Material
Hardware Interface
Supported Software
Speaker Count
Output Wattage
Wireless Microphone Frequency
GSM frequencies
Color Screen
Voltage
Pricing
Binding
Rated
Is Assembly Required?
Manufacturer recommended age
Release date
Mfg Recom

In [520]:
#Creating Final dataset
final = m_1[['name','ASIN', 'Best Sellers Rank', 'Manufacturer', 'Product Dimensions','Package Dimensions','Item Weight','Date First Available','Is Discontinued By Manufacturer', 'Item model number','Batteries','Voltage',  'Standing screen display size',  'Colour', 'Color Name',   ]]
final.head()

,name,ASIN,Best Sellers Rank,Manufacturer,Product Dimensions,Package Dimensions,Item Weight,Date First Available,Is Discontinued By Manufacturer,Item model number,Batteries,Voltage,Standing screen display size,Colour,Color Name
0,Simple Mobile Samsung Galaxy A01 4G LTE Prepai...,B088N774B7,"#3,750 in Cell Phones & Accessories ( See Top ...",Simple Mobile,6.75 x 2.5 x 10.38 inches,NaN,14.4 ounces,"May 13, 2020",NaN,SMSAS111DGP5,1 Lithium ion batteries required. (included),NaN,NaN,NaN,NaN
1,Simple Mobile Samsung Galaxy A20 4G LTE Prepai...,B07Y5V9CSW,"#45,762 in Cell Phones & Accessories ( See Top...",Simple Mobile,7.25 x 1.65 x 10.25 inches,NaN,15.2 ounces,"September 20, 2019",NaN,SMSAS205DGP5,1 Lithium ion batteries required. (included),NaN,NaN,NaN,NaN
2,None,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"OUKITEL WP8 Pro (2020) Rugged Smartphone, 6.49...",B08HRJM99F,"#9,229 in Cell Phones & Accessories ( See Top ...",OUKITEL,7.01 x 3.33 x 0.51 inches,NaN,1.23 pounds,"October 22, 2020",NaN,WP8 Pro,1 Lithium Polymer batteries required. (included),NaN,NaN,Orange,NaN
4,Samsung Galaxy S20 FE 5G | Factory Unlocked An...,B08FYTSXGQ,#491 in Cell Phones & Accessories ( See Top 10...,Samsung,6.7 x 3.4 x 2.44 inches,NaN,6.7 ounces,"September 23, 2020",NaN,SM-G781UZBMXAA,1 Lithium ion batteries required. (included),NaN,NaN,Cloud Navy,NaN


In [521]:
#Dropping duplicates from final dataset
final.drop_duplicates(subset ="ASIN",keep = False, inplace = True) 
final.head()

<ipython-input-521-868a276802f6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final.drop_duplicates(subset ="ASIN",keep = False, inplace = True)


,name,ASIN,Best Sellers Rank,Manufacturer,Product Dimensions,Package Dimensions,Item Weight,Date First Available,Is Discontinued By Manufacturer,Item model number,Batteries,Voltage,Standing screen display size,Colour,Color Name
3,"OUKITEL WP8 Pro (2020) Rugged Smartphone, 6.49...",B08HRJM99F,"#9,229 in Cell Phones & Accessories ( See Top ...",OUKITEL,7.01 x 3.33 x 0.51 inches,NaN,1.23 pounds,"October 22, 2020",NaN,WP8 Pro,1 Lithium Polymer batteries required. (included),NaN,NaN,Orange,NaN
4,Samsung Galaxy S20 FE 5G | Factory Unlocked An...,B08FYTSXGQ,#491 in Cell Phones & Accessories ( See Top 10...,Samsung,6.7 x 3.4 x 2.44 inches,NaN,6.7 ounces,"September 23, 2020",NaN,SM-G781UZBMXAA,1 Lithium ion batteries required. (included),NaN,NaN,Cloud Navy,NaN
5,"BLU G90 Pro – 6.5” HD+ Gaming Smartphone, Quad...",B08C6XBCRT,"#1,506 in Cell Phones & Accessories ( See Top ...",BLU,15.99 x 8.99 x 1 inches,NaN,0.023 ounces,"July 1, 2020",NaN,G0310WW,1 Lithium ion batteries required. (included),NaN,NaN,NaN,NaN
8,"TCL 10L, Unlocked Android Smartphone with 6.53...",B087LY84RT,"#4,182 in Cell Phones & Accessories ( See Top ...",TCL,6.38 x 2.98 x 0.33 inches,NaN,12 ounces,"April 22, 2020",NaN,TCL-T770B-2ALCUS11-2,1 Lithium ion batteries required. (included),NaN,NaN,Arctic White,NaN
10,"OUKITEL WP5 (2020) Rugged Smartphone, 8000mAh ...",B0836TG4XB,"#12,321 in Cell Phones & Accessories ( See Top...",OUKITEL,6.06 x 2.92 x 0.71 inches,NaN,9.8 ounces,"December 27, 2019",NaN,WP5,1 Lithium Polymer batteries required. (included),NaN,NaN,Black,NaN


In [522]:
#Resetting Index
final = final.reset_index(drop= True)
final.head()

,name,ASIN,Best Sellers Rank,Manufacturer,Product Dimensions,Package Dimensions,Item Weight,Date First Available,Is Discontinued By Manufacturer,Item model number,Batteries,Voltage,Standing screen display size,Colour,Color Name
0,"OUKITEL WP8 Pro (2020) Rugged Smartphone, 6.49...",B08HRJM99F,"#9,229 in Cell Phones & Accessories ( See Top ...",OUKITEL,7.01 x 3.33 x 0.51 inches,NaN,1.23 pounds,"October 22, 2020",NaN,WP8 Pro,1 Lithium Polymer batteries required. (included),NaN,NaN,Orange,NaN
1,Samsung Galaxy S20 FE 5G | Factory Unlocked An...,B08FYTSXGQ,#491 in Cell Phones & Accessories ( See Top 10...,Samsung,6.7 x 3.4 x 2.44 inches,NaN,6.7 ounces,"September 23, 2020",NaN,SM-G781UZBMXAA,1 Lithium ion batteries required. (included),NaN,NaN,Cloud Navy,NaN
2,"BLU G90 Pro – 6.5” HD+ Gaming Smartphone, Quad...",B08C6XBCRT,"#1,506 in Cell Phones & Accessories ( See Top ...",BLU,15.99 x 8.99 x 1 inches,NaN,0.023 ounces,"July 1, 2020",NaN,G0310WW,1 Lithium ion batteries required. (included),NaN,NaN,NaN,NaN
3,"TCL 10L, Unlocked Android Smartphone with 6.53...",B087LY84RT,"#4,182 in Cell Phones & Accessories ( See Top ...",TCL,6.38 x 2.98 x 0.33 inches,NaN,12 ounces,"April 22, 2020",NaN,TCL-T770B-2ALCUS11-2,1 Lithium ion batteries required. (included),NaN,NaN,Arctic White,NaN
4,"OUKITEL WP5 (2020) Rugged Smartphone, 8000mAh ...",B0836TG4XB,"#12,321 in Cell Phones & Accessories ( See Top...",OUKITEL,6.06 x 2.92 x 0.71 inches,NaN,9.8 ounces,"December 27, 2019",NaN,WP5,1 Lithium Polymer batteries required. (included),NaN,NaN,Black,NaN


In [523]:
final['Product Dimensions (inches)'] = final['Product Dimensions'].str.rstrip(' inches')
final['Product Dimensions (inches)']

0       7.01 x 3.33 x 0.51
1         6.7 x 3.4 x 2.44
2         15.99 x 8.99 x 1
3       6.38 x 2.98 x 0.33
4       6.06 x 2.92 x 0.71
               ...        
2720             2 x 6 x 3
2721    5.91 x 4.72 x 2.36
2722                   NaN
2723                   NaN
2724                   NaN
Name: Product Dimensions (inches), Length: 2725, dtype: object

In [524]:
final['Product Dimensions'] = final['Product Dimensions (inches)']
final.head()

,name,ASIN,Best Sellers Rank,Manufacturer,Product Dimensions,Package Dimensions,Item Weight,Date First Available,Is Discontinued By Manufacturer,Item model number,Batteries,Voltage,Standing screen display size,Colour,Color Name,Product Dimensions (inches)
0,"OUKITEL WP8 Pro (2020) Rugged Smartphone, 6.49...",B08HRJM99F,"#9,229 in Cell Phones & Accessories ( See Top ...",OUKITEL,7.01 x 3.33 x 0.51,NaN,1.23 pounds,"October 22, 2020",NaN,WP8 Pro,1 Lithium Polymer batteries required. (included),NaN,NaN,Orange,NaN,7.01 x 3.33 x 0.51
1,Samsung Galaxy S20 FE 5G | Factory Unlocked An...,B08FYTSXGQ,#491 in Cell Phones & Accessories ( See Top 10...,Samsung,6.7 x 3.4 x 2.44,NaN,6.7 ounces,"September 23, 2020",NaN,SM-G781UZBMXAA,1 Lithium ion batteries required. (included),NaN,NaN,Cloud Navy,NaN,6.7 x 3.4 x 2.44
2,"BLU G90 Pro – 6.5” HD+ Gaming Smartphone, Quad...",B08C6XBCRT,"#1,506 in Cell Phones & Accessories ( See Top ...",BLU,15.99 x 8.99 x 1,NaN,0.023 ounces,"July 1, 2020",NaN,G0310WW,1 Lithium ion batteries required. (included),NaN,NaN,NaN,NaN,15.99 x 8.99 x 1
3,"TCL 10L, Unlocked Android Smartphone with 6.53...",B087LY84RT,"#4,182 in Cell Phones & Accessories ( See Top ...",TCL,6.38 x 2.98 x 0.33,NaN,12 ounces,"April 22, 2020",NaN,TCL-T770B-2ALCUS11-2,1 Lithium ion batteries required. (included),NaN,NaN,Arctic White,NaN,6.38 x 2.98 x 0.33
4,"OUKITEL WP5 (2020) Rugged Smartphone, 8000mAh ...",B0836TG4XB,"#12,321 in Cell Phones & Accessories ( See Top...",OUKITEL,6.06 x 2.92 x 0.71,NaN,9.8 ounces,"December 27, 2019",NaN,WP5,1 Lithium Polymer batteries required. (included),NaN,NaN,Black,NaN,6.06 x 2.92 x 0.71


In [525]:
#Dropping Product Dimensions from the dataset
final = final.drop(['Product Dimensions (inches)'], axis=1)
final.head()

,name,ASIN,Best Sellers Rank,Manufacturer,Product Dimensions,Package Dimensions,Item Weight,Date First Available,Is Discontinued By Manufacturer,Item model number,Batteries,Voltage,Standing screen display size,Colour,Color Name
0,"OUKITEL WP8 Pro (2020) Rugged Smartphone, 6.49...",B08HRJM99F,"#9,229 in Cell Phones & Accessories ( See Top ...",OUKITEL,7.01 x 3.33 x 0.51,NaN,1.23 pounds,"October 22, 2020",NaN,WP8 Pro,1 Lithium Polymer batteries required. (included),NaN,NaN,Orange,NaN
1,Samsung Galaxy S20 FE 5G | Factory Unlocked An...,B08FYTSXGQ,#491 in Cell Phones & Accessories ( See Top 10...,Samsung,6.7 x 3.4 x 2.44,NaN,6.7 ounces,"September 23, 2020",NaN,SM-G781UZBMXAA,1 Lithium ion batteries required. (included),NaN,NaN,Cloud Navy,NaN
2,"BLU G90 Pro – 6.5” HD+ Gaming Smartphone, Quad...",B08C6XBCRT,"#1,506 in Cell Phones & Accessories ( See Top ...",BLU,15.99 x 8.99 x 1,NaN,0.023 ounces,"July 1, 2020",NaN,G0310WW,1 Lithium ion batteries required. (included),NaN,NaN,NaN,NaN
3,"TCL 10L, Unlocked Android Smartphone with 6.53...",B087LY84RT,"#4,182 in Cell Phones & Accessories ( See Top ...",TCL,6.38 x 2.98 x 0.33,NaN,12 ounces,"April 22, 2020",NaN,TCL-T770B-2ALCUS11-2,1 Lithium ion batteries required. (included),NaN,NaN,Arctic White,NaN
4,"OUKITEL WP5 (2020) Rugged Smartphone, 8000mAh ...",B0836TG4XB,"#12,321 in Cell Phones & Accessories ( See Top...",OUKITEL,6.06 x 2.92 x 0.71,NaN,9.8 ounces,"December 27, 2019",NaN,WP5,1 Lithium Polymer batteries required. (included),NaN,NaN,Black,NaN


In [526]:
#Splitting Item Weight column as some weights are in ounces while some are in pound
a = final['Item Weight'].str.split(expand=True)
a.columns = ['a', 'b']
a

,a,b
0,1.23,pounds
1,6.7,ounces
2,0.023,ounces
3,12,ounces
4,9.8,ounces
...,...,...
2720,1,ounces
2721,0.106,ounces
2722,1.36,pounds
2723,2.39,ounces


In [527]:
#Converting ounces weight in pounds to have similar data in 'Item Weight' column. 
a.a = a['a'].astype(float)
a['a'] = np.where(a['b'] == 'ounces',a['a'] * 0.16,a['a'])
final['Item Weight'] = a['a']
final.head()

,name,ASIN,Best Sellers Rank,Manufacturer,Product Dimensions,Package Dimensions,Item Weight,Date First Available,Is Discontinued By Manufacturer,Item model number,Batteries,Voltage,Standing screen display size,Colour,Color Name
0,"OUKITEL WP8 Pro (2020) Rugged Smartphone, 6.49...",B08HRJM99F,"#9,229 in Cell Phones & Accessories ( See Top ...",OUKITEL,7.01 x 3.33 x 0.51,NaN,1.23000,"October 22, 2020",NaN,WP8 Pro,1 Lithium Polymer batteries required. (included),NaN,NaN,Orange,NaN
1,Samsung Galaxy S20 FE 5G | Factory Unlocked An...,B08FYTSXGQ,#491 in Cell Phones & Accessories ( See Top 10...,Samsung,6.7 x 3.4 x 2.44,NaN,1.07200,"September 23, 2020",NaN,SM-G781UZBMXAA,1 Lithium ion batteries required. (included),NaN,NaN,Cloud Navy,NaN
2,"BLU G90 Pro – 6.5” HD+ Gaming Smartphone, Quad...",B08C6XBCRT,"#1,506 in Cell Phones & Accessories ( See Top ...",BLU,15.99 x 8.99 x 1,NaN,0.00368,"July 1, 2020",NaN,G0310WW,1 Lithium ion batteries required. (included),NaN,NaN,NaN,NaN
3,"TCL 10L, Unlocked Android Smartphone with 6.53...",B087LY84RT,"#4,182 in Cell Phones & Accessories ( See Top ...",TCL,6.38 x 2.98 x 0.33,NaN,1.92000,"April 22, 2020",NaN,TCL-T770B-2ALCUS11-2,1 Lithium ion batteries required. (included),NaN,NaN,Arctic White,NaN
4,"OUKITEL WP5 (2020) Rugged Smartphone, 8000mAh ...",B0836TG4XB,"#12,321 in Cell Phones & Accessories ( See Top...",OUKITEL,6.06 x 2.92 x 0.71,NaN,1.56800,"December 27, 2019",NaN,WP5,1 Lithium Polymer batteries required. (included),NaN,NaN,Black,NaN


In [528]:
#Renaming columns along with their unit values
final = final.rename(columns={'Product Dimensions' : 'Product Dimensions (inches)', 'Item Weight':'Item Weight (pounds)'})
final.head()

,name,ASIN,Best Sellers Rank,Manufacturer,Product Dimensions (inches),Package Dimensions,Item Weight (pounds),Date First Available,Is Discontinued By Manufacturer,Item model number,Batteries,Voltage,Standing screen display size,Colour,Color Name
0,"OUKITEL WP8 Pro (2020) Rugged Smartphone, 6.49...",B08HRJM99F,"#9,229 in Cell Phones & Accessories ( See Top ...",OUKITEL,7.01 x 3.33 x 0.51,NaN,1.23000,"October 22, 2020",NaN,WP8 Pro,1 Lithium Polymer batteries required. (included),NaN,NaN,Orange,NaN
1,Samsung Galaxy S20 FE 5G | Factory Unlocked An...,B08FYTSXGQ,#491 in Cell Phones & Accessories ( See Top 10...,Samsung,6.7 x 3.4 x 2.44,NaN,1.07200,"September 23, 2020",NaN,SM-G781UZBMXAA,1 Lithium ion batteries required. (included),NaN,NaN,Cloud Navy,NaN
2,"BLU G90 Pro – 6.5” HD+ Gaming Smartphone, Quad...",B08C6XBCRT,"#1,506 in Cell Phones & Accessories ( See Top ...",BLU,15.99 x 8.99 x 1,NaN,0.00368,"July 1, 2020",NaN,G0310WW,1 Lithium ion batteries required. (included),NaN,NaN,NaN,NaN
3,"TCL 10L, Unlocked Android Smartphone with 6.53...",B087LY84RT,"#4,182 in Cell Phones & Accessories ( See Top ...",TCL,6.38 x 2.98 x 0.33,NaN,1.92000,"April 22, 2020",NaN,TCL-T770B-2ALCUS11-2,1 Lithium ion batteries required. (included),NaN,NaN,Arctic White,NaN
4,"OUKITEL WP5 (2020) Rugged Smartphone, 8000mAh ...",B0836TG4XB,"#12,321 in Cell Phones & Accessories ( See Top...",OUKITEL,6.06 x 2.92 x 0.71,NaN,1.56800,"December 27, 2019",NaN,WP5,1 Lithium Polymer batteries required. (included),NaN,NaN,Black,NaN


In [529]:
#Fetching only Best Seller Rand from 'Best Seller Rank' column. 
final['Best_Sellers_Rank_fields'] = final['Best Sellers Rank'].str.split('(').str[0]

In [530]:
final['Best Sellers Rank'] = final['Best_Sellers_Rank_fields']

In [531]:
final['Best_Sellers_Rank_1'] = final['Best Sellers Rank'].str.split(' in').str[0]

In [532]:
final['Best Sellers Rank'] = final['Best_Sellers_Rank_1']
final

,name,ASIN,Best Sellers Rank,Manufacturer,Product Dimensions (inches),Package Dimensions,Item Weight (pounds),Date First Available,Is Discontinued By Manufacturer,Item model number,Batteries,Voltage,Standing screen display size,Colour,Color Name,Best_Sellers_Rank_fields,Best_Sellers_Rank_1
0,"OUKITEL WP8 Pro (2020) Rugged Smartphone, 6.49...",B08HRJM99F,"#9,229",OUKITEL,7.01 x 3.33 x 0.51,NaN,1.23000,"October 22, 2020",NaN,WP8 Pro,1 Lithium Polymer batteries required. (included),NaN,NaN,Orange,NaN,"#9,229 in Cell Phones & Accessories","#9,229"
1,Samsung Galaxy S20 FE 5G | Factory Unlocked An...,B08FYTSXGQ,#491,Samsung,6.7 x 3.4 x 2.44,NaN,1.07200,"September 23, 2020",NaN,SM-G781UZBMXAA,1 Lithium ion batteries required. (included),NaN,NaN,Cloud Navy,NaN,#491 in Cell Phones & Accessories,#491
2,"BLU G90 Pro – 6.5” HD+ Gaming Smartphone, Quad...",B08C6XBCRT,"#1,506",BLU,15.99 x 8.99 x 1,NaN,0.00368,"July 1, 2020",NaN,G0310WW,1 Lithium ion batteries required. (included),NaN,NaN,NaN,NaN,"#1,506 in Cell Phones & Accessories","#1,506"
3,"TCL 10L, Unlocked Android Smartphone with 6.53...",B087LY84RT,"#4,182",TCL,6.38 x 2.98 x 0.33,NaN,1.92000,"April 22, 2020",NaN,TCL-T770B-2ALCUS11-2,1 Lithium ion batteries required. (included),NaN,NaN,Arctic White,NaN,"#4,182 in Cell Phones & Accessories","#4,182"
4,"OUKITEL WP5 (2020) Rugged Smartphone, 8000mAh ...",B0836TG4XB,"#12,321",OUKITEL,6.06 x 2.92 x 0.71,NaN,1.56800,"December 27, 2019",NaN,WP5,1 Lithium Polymer batteries required. (included),NaN,NaN,Black,NaN,"#12,321 in Cell Phones & Accessories","#12,321"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2720,GEESHIDA 4 Pack Earbud Headphones in-Ear mic 3...,B07P2RQVFL,"#5,148",Geeboy,2 x 6 x 3,NaN,0.16000,"February 24, 2019",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"#5,148 in Earbud & In-Ear Headphones","#5,148"
2721,Blips - New Lab Kit 2 - Transform Your Smartph...,B07YWXZPFQ,"#6,571",blips magnify the world,5.91 x 4.72 x 2.36,NaN,0.01696,"November 23, 2019",NaN,NaN,2 Lithium Metal batteries required. (included),NaN,NaN,NaN,NaN,"#6,571 in Camera & Photo","#6,571"
2722,Godox X2T-S 2.4G TTL Wireless Flash Trigger wi...,B07W41VP3B,#64,Godox,NaN,7.72 x 5.16 x 4.49 inches,1.36000,"July 25, 2019",NaN,NaN,NaN,NaN,NaN,NaN,NaN,#64 in Photographic Lighting Remote Triggers,#64
2723,"Car Phone Mount,Cell Phone Holder for Car,Anti...",B08MT72RFJ,"#97,438",Senhorita,NaN,4.9 x 3.9 x 2 inches,0.38240,"November 5, 2020",NaN,NaN,NaN,NaN,NaN,Original,NaN,"#97,438 in Cell Phones & Accessories","#97,438"


In [533]:
final['Best_Sellers_Rank_fields_1'] = final['Best_Sellers_Rank_fields'].str.split('in ').str[1]

In [534]:
final['Best_Sellers_Rank_fields'] = final['Best_Sellers_Rank_fields_1']

In [535]:
final.head()

,name,ASIN,Best Sellers Rank,Manufacturer,Product Dimensions (inches),Package Dimensions,Item Weight (pounds),Date First Available,Is Discontinued By Manufacturer,Item model number,Batteries,Voltage,Standing screen display size,Colour,Color Name,Best_Sellers_Rank_fields,Best_Sellers_Rank_1,Best_Sellers_Rank_fields_1
0,"OUKITEL WP8 Pro (2020) Rugged Smartphone, 6.49...",B08HRJM99F,"#9,229",OUKITEL,7.01 x 3.33 x 0.51,NaN,1.23000,"October 22, 2020",NaN,WP8 Pro,1 Lithium Polymer batteries required. (included),NaN,NaN,Orange,NaN,Cell Phones & Accessories,"#9,229",Cell Phones & Accessories
1,Samsung Galaxy S20 FE 5G | Factory Unlocked An...,B08FYTSXGQ,#491,Samsung,6.7 x 3.4 x 2.44,NaN,1.07200,"September 23, 2020",NaN,SM-G781UZBMXAA,1 Lithium ion batteries required. (included),NaN,NaN,Cloud Navy,NaN,Cell Phones & Accessories,#491,Cell Phones & Accessories
2,"BLU G90 Pro – 6.5” HD+ Gaming Smartphone, Quad...",B08C6XBCRT,"#1,506",BLU,15.99 x 8.99 x 1,NaN,0.00368,"July 1, 2020",NaN,G0310WW,1 Lithium ion batteries required. (included),NaN,NaN,NaN,NaN,Cell Phones & Accessories,"#1,506",Cell Phones & Accessories
3,"TCL 10L, Unlocked Android Smartphone with 6.53...",B087LY84RT,"#4,182",TCL,6.38 x 2.98 x 0.33,NaN,1.92000,"April 22, 2020",NaN,TCL-T770B-2ALCUS11-2,1 Lithium ion batteries required. (included),NaN,NaN,Arctic White,NaN,Cell Phones & Accessories,"#4,182",Cell Phones & Accessories
4,"OUKITEL WP5 (2020) Rugged Smartphone, 8000mAh ...",B0836TG4XB,"#12,321",OUKITEL,6.06 x 2.92 x 0.71,NaN,1.56800,"December 27, 2019",NaN,WP5,1 Lithium Polymer batteries required. (included),NaN,NaN,Black,NaN,Cell Phones & Accessories,"#12,321",Cell Phones & Accessories


In [536]:
#Dropping few columns which are not required
final.drop(['Best_Sellers_Rank_1', 'Best_Sellers_Rank_fields_1'], axis=1, inplace=True)

In [537]:
cols_b1= final.columns.tolist()
cols_b1

['name',
 'ASIN',
 'Best Sellers Rank',
 'Manufacturer',
 'Product Dimensions (inches)',
 'Package Dimensions',
 'Item Weight (pounds)',
 'Date First Available',
 'Is Discontinued By Manufacturer',
 'Item model number',
 'Batteries',
 'Voltage',
 'Standing screen display size',
 'Colour',
 'Color Name',
 'Best_Sellers_Rank_fields']

In [538]:
cols_b1= cols_b1[:3] + cols_b1[-1:] + cols_b1[3:-1]
cols_b1

['name',
 'ASIN',
 'Best Sellers Rank',
 'Best_Sellers_Rank_fields',
 'Manufacturer',
 'Product Dimensions (inches)',
 'Package Dimensions',
 'Item Weight (pounds)',
 'Date First Available',
 'Is Discontinued By Manufacturer',
 'Item model number',
 'Batteries',
 'Voltage',
 'Standing screen display size',
 'Colour',
 'Color Name']

In [539]:
final = final[cols_b1]
final.head()

,name,ASIN,Best Sellers Rank,Best_Sellers_Rank_fields,Manufacturer,Product Dimensions (inches),Package Dimensions,Item Weight (pounds),Date First Available,Is Discontinued By Manufacturer,Item model number,Batteries,Voltage,Standing screen display size,Colour,Color Name
0,"OUKITEL WP8 Pro (2020) Rugged Smartphone, 6.49...",B08HRJM99F,"#9,229",Cell Phones & Accessories,OUKITEL,7.01 x 3.33 x 0.51,NaN,1.23000,"October 22, 2020",NaN,WP8 Pro,1 Lithium Polymer batteries required. (included),NaN,NaN,Orange,NaN
1,Samsung Galaxy S20 FE 5G | Factory Unlocked An...,B08FYTSXGQ,#491,Cell Phones & Accessories,Samsung,6.7 x 3.4 x 2.44,NaN,1.07200,"September 23, 2020",NaN,SM-G781UZBMXAA,1 Lithium ion batteries required. (included),NaN,NaN,Cloud Navy,NaN
2,"BLU G90 Pro – 6.5” HD+ Gaming Smartphone, Quad...",B08C6XBCRT,"#1,506",Cell Phones & Accessories,BLU,15.99 x 8.99 x 1,NaN,0.00368,"July 1, 2020",NaN,G0310WW,1 Lithium ion batteries required. (included),NaN,NaN,NaN,NaN
3,"TCL 10L, Unlocked Android Smartphone with 6.53...",B087LY84RT,"#4,182",Cell Phones & Accessories,TCL,6.38 x 2.98 x 0.33,NaN,1.92000,"April 22, 2020",NaN,TCL-T770B-2ALCUS11-2,1 Lithium ion batteries required. (included),NaN,NaN,Arctic White,NaN
4,"OUKITEL WP5 (2020) Rugged Smartphone, 8000mAh ...",B0836TG4XB,"#12,321",Cell Phones & Accessories,OUKITEL,6.06 x 2.92 x 0.71,NaN,1.56800,"December 27, 2019",NaN,WP5,1 Lithium Polymer batteries required. (included),NaN,NaN,Black,NaN


In [540]:
#Removing Hash and commas from Best Seller Rank
final['Best Sellers Rank'] = final['Best Sellers Rank'].str.replace('#', '').str.replace(',', '')

In [541]:
final.head()

,name,ASIN,Best Sellers Rank,Best_Sellers_Rank_fields,Manufacturer,Product Dimensions (inches),Package Dimensions,Item Weight (pounds),Date First Available,Is Discontinued By Manufacturer,Item model number,Batteries,Voltage,Standing screen display size,Colour,Color Name
0,"OUKITEL WP8 Pro (2020) Rugged Smartphone, 6.49...",B08HRJM99F,9229,Cell Phones & Accessories,OUKITEL,7.01 x 3.33 x 0.51,NaN,1.23000,"October 22, 2020",NaN,WP8 Pro,1 Lithium Polymer batteries required. (included),NaN,NaN,Orange,NaN
1,Samsung Galaxy S20 FE 5G | Factory Unlocked An...,B08FYTSXGQ,491,Cell Phones & Accessories,Samsung,6.7 x 3.4 x 2.44,NaN,1.07200,"September 23, 2020",NaN,SM-G781UZBMXAA,1 Lithium ion batteries required. (included),NaN,NaN,Cloud Navy,NaN
2,"BLU G90 Pro – 6.5” HD+ Gaming Smartphone, Quad...",B08C6XBCRT,1506,Cell Phones & Accessories,BLU,15.99 x 8.99 x 1,NaN,0.00368,"July 1, 2020",NaN,G0310WW,1 Lithium ion batteries required. (included),NaN,NaN,NaN,NaN
3,"TCL 10L, Unlocked Android Smartphone with 6.53...",B087LY84RT,4182,Cell Phones & Accessories,TCL,6.38 x 2.98 x 0.33,NaN,1.92000,"April 22, 2020",NaN,TCL-T770B-2ALCUS11-2,1 Lithium ion batteries required. (included),NaN,NaN,Arctic White,NaN
4,"OUKITEL WP5 (2020) Rugged Smartphone, 8000mAh ...",B0836TG4XB,12321,Cell Phones & Accessories,OUKITEL,6.06 x 2.92 x 0.71,NaN,1.56800,"December 27, 2019",NaN,WP5,1 Lithium Polymer batteries required. (included),NaN,NaN,Black,NaN


In [542]:
#Quick check on Null Values
final.isnull().sum()

name                                  0
ASIN                                  0
Best Sellers Rank                    27
Best_Sellers_Rank_fields             27
Manufacturer                        214
Product Dimensions (inches)        1463
Package Dimensions                 1450
Item Weight (pounds)                148
Date First Available                 22
Is Discontinued By Manufacturer    1952
Item model number                  1596
Batteries                          2102
Voltage                            2723
Standing screen display size       2724
Colour                             1673
Color Name                         2686
dtype: int64

In [543]:
final['Package Dimensions (inches)'] = final['Package Dimensions'].str.rstrip(' inches')
final['Package Dimensions'] = final['Package Dimensions (inches)']
final = final.drop(['Package Dimensions (inches)'], axis=1)

In [544]:
final = final.rename(columns={'Package Dimensions' : 'Package Dimensions (inches)'})
final

,name,ASIN,Best Sellers Rank,Best_Sellers_Rank_fields,Manufacturer,Product Dimensions (inches),Package Dimensions (inches),Item Weight (pounds),Date First Available,Is Discontinued By Manufacturer,Item model number,Batteries,Voltage,Standing screen display size,Colour,Color Name
0,"OUKITEL WP8 Pro (2020) Rugged Smartphone, 6.49...",B08HRJM99F,9229,Cell Phones & Accessories,OUKITEL,7.01 x 3.33 x 0.51,NaN,1.23000,"October 22, 2020",NaN,WP8 Pro,1 Lithium Polymer batteries required. (included),NaN,NaN,Orange,NaN
1,Samsung Galaxy S20 FE 5G | Factory Unlocked An...,B08FYTSXGQ,491,Cell Phones & Accessories,Samsung,6.7 x 3.4 x 2.44,NaN,1.07200,"September 23, 2020",NaN,SM-G781UZBMXAA,1 Lithium ion batteries required. (included),NaN,NaN,Cloud Navy,NaN
2,"BLU G90 Pro – 6.5” HD+ Gaming Smartphone, Quad...",B08C6XBCRT,1506,Cell Phones & Accessories,BLU,15.99 x 8.99 x 1,NaN,0.00368,"July 1, 2020",NaN,G0310WW,1 Lithium ion batteries required. (included),NaN,NaN,NaN,NaN
3,"TCL 10L, Unlocked Android Smartphone with 6.53...",B087LY84RT,4182,Cell Phones & Accessories,TCL,6.38 x 2.98 x 0.33,NaN,1.92000,"April 22, 2020",NaN,TCL-T770B-2ALCUS11-2,1 Lithium ion batteries required. (included),NaN,NaN,Arctic White,NaN
4,"OUKITEL WP5 (2020) Rugged Smartphone, 8000mAh ...",B0836TG4XB,12321,Cell Phones & Accessories,OUKITEL,6.06 x 2.92 x 0.71,NaN,1.56800,"December 27, 2019",NaN,WP5,1 Lithium Polymer batteries required. (included),NaN,NaN,Black,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2720,GEESHIDA 4 Pack Earbud Headphones in-Ear mic 3...,B07P2RQVFL,5148,Earbud & In-Ear Headphones,Geeboy,2 x 6 x 3,NaN,0.16000,"February 24, 2019",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2721,Blips - New Lab Kit 2 - Transform Your Smartph...,B07YWXZPFQ,6571,Camera & Photo,blips magnify the world,5.91 x 4.72 x 2.36,NaN,0.01696,"November 23, 2019",NaN,NaN,2 Lithium Metal batteries required. (included),NaN,NaN,NaN,NaN
2722,Godox X2T-S 2.4G TTL Wireless Flash Trigger wi...,B07W41VP3B,64,Photographic Lighting Remote Triggers,Godox,NaN,7.72 x 5.16 x 4.49,1.36000,"July 25, 2019",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2723,"Car Phone Mount,Cell Phone Holder for Car,Anti...",B08MT72RFJ,97438,Cell Phones & Accessories,Senhorita,NaN,4.9 x 3.9 x 2,0.38240,"November 5, 2020",NaN,NaN,NaN,NaN,NaN,Original,NaN


In [545]:
final['Date First Available'] =  pd.to_datetime(final['Date First Available'], infer_datetime_format=True)

In [546]:
#Dropping Color Name' column
final = final.drop(columns= 'Color Name', axis=1)

In [547]:
final.iloc[30]

name                               OUKITEL WP8 Pro Unlocked Rugged Smartphone, An...
ASIN                                                                      B08PFGPGRM
Best Sellers Rank                                                              20585
Best_Sellers_Rank_fields                                  Cell Phones & Accessories 
Manufacturer                                                    OUKITEL direct store
Product Dimensions (inches)                                                      NaN
Package Dimensions (inches)                                       9.53 x 8.66 x 1.81
Item Weight (pounds)                                                            1.24
Date First Available                                             2020-12-02 00:00:00
Is Discontinued By Manufacturer                                                  NaN
Item model number                                                                NaN
Batteries                                                        

In [548]:
final.dtypes

name                                       object
ASIN                                       object
Best Sellers Rank                          object
Best_Sellers_Rank_fields                   object
Manufacturer                               object
Product Dimensions (inches)                object
Package Dimensions (inches)                object
Item Weight (pounds)                      float64
Date First Available               datetime64[ns]
Is Discontinued By Manufacturer            object
Item model number                          object
Batteries                                  object
Voltage                                    object
Standing screen display size               object
Colour                                     object
dtype: object

In [549]:
final['Product Dimensions (inches)'].fillna(final['Package Dimensions (inches)'], inplace=True)
del final['Package Dimensions (inches)']


In [550]:
final.isnull().sum()

name                                  0
ASIN                                  0
Best Sellers Rank                    27
Best_Sellers_Rank_fields             27
Manufacturer                        214
Product Dimensions (inches)         188
Item Weight (pounds)                148
Date First Available                 22
Is Discontinued By Manufacturer    1952
Item model number                  1596
Batteries                          2102
Voltage                            2723
Standing screen display size       2724
Colour                             1673
dtype: int64

In [551]:
#Filling Null Values in Best Seller Rank Column
final['Best Sellers Rank'].fillna(0, inplace= True)

In [552]:
#Converting Best Sellers Rank as float
final['Best Sellers Rank'] = final['Best Sellers Rank'].astype('float')

In [553]:
#Deleting few
del final['Voltage']

In [554]:
del final['Standing screen display size']

In [555]:
#Saving cleaned data to CSV for SQL Analysis
final.to_csv('product_smartphone.csv')

In [ ]:
#Pushing everything to MySQL for further analysis

In [556]:
# db_data = 'mysql+mysqldb://' + 'root' + ':' + '234024_Anup' + '@' + 'localhost' + ':3306/' \
#        + 'smartphone' + '?charset=utf8mb4'
# engine = create_engine(db_data)

# # Connect to the database
# connection = pymysql.connect(host='localhost',
#                          user='root',
#                          password='234024_Anup',
#                          db='smartphone')    

In [558]:
#connection

In [1182]:
# # create cursor
# cursor=connection.cursor()
# # Execute the to_sql for writting DF into SQL
# new_f3.to_sql('search', engine, if_exists='append', index=False)    

# # Execute query
# sql = "SELECT * FROM search"
# cursor.execute(sql)

# # Fetch all the records
# result = cursor.fetchall()
# for i in result:
#     print(i)

# engine.dispose()
# connection.close()